# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv_path = os.path.join("..","output_data","cities.csv")

cities_df = pd.read_csv(cities_csv_path)
cities_df

,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Wind Speed
0,ribeira grande,PT,100.0,1579999970,73,38.52,-28.70,54.55,54.55,4.36
1,dikson,RU,100.0,1580000107,97,73.51,80.55,-7.56,-7.56,6.60
2,koygorodok,RU,100.0,1580000108,96,60.45,51.00,7.50,7.50,2.13
3,jamestown,SH,95.0,1580000108,75,-15.94,-5.72,74.93,74.93,14.74
4,prieska,ZA,0.0,1580000109,47,-29.66,22.75,69.51,69.51,10.13
...,...,...,...,...,...,...,...,...,...,...
546,krasnyy chikoy,RU,0.0,1580000364,84,50.36,108.75,0.27,0.27,1.39
547,ha giang,VN,71.0,1580000364,85,22.83,104.98,44.74,44.74,3.51
548,tateyama,JP,75.0,1580000365,86,34.98,139.87,39.99,37.00,4.70
549,khash,IR,0.0,1580000365,74,28.22,61.22,43.75,43.75,3.94


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Plot Heatmap
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat","Lng"]]
fig = gmaps.figure(zoom_level = 2,
                   center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=cities_df["Humidity"], 
                                 dissipating=False, 
                                 max_intensity=200,
                                 point_radius=5)

heat_layer.gradient = [
    (255, 255, 50, 0.3),
    (100,100,100,.03),
    (0, 200, 255, 0.9)
]



# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# A max temperature lower than 80 degrees but higher than 70.
ideal_cities = cities_df.loc[cities_df["Min Temp"] < 90 ]
ideal_cities = ideal_cities.loc[ideal_cities["Min Temp"] > 70]
# Wind speed less than 10 mph.
ideal_cities = ideal_cities.loc[ideal_cities["Wind Speed"] < 10 ]
# Zero cloudiness.
ideal_cities = ideal_cities.loc[ideal_cities["Cloudiness"] == 0]

ideal_cities.dropna(inplace = True)
ideal_cities


,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Wind Speed
31,arraial do cabo,BR,0.0,1579999938,82,-22.97,-42.03,75.78,75.78,8.32
120,takapau,NZ,0.0,1580000163,68,-40.03,176.35,84.00,75.99,4.00
147,port macquarie,AU,0.0,1580000177,55,-31.43,152.92,97.00,86.00,5.82
193,karratha,AU,0.0,1580000199,74,-20.74,116.85,85.17,85.17,4.34
295,bella vista,PY,0.0,1580000248,53,-22.13,-56.52,74.61,74.61,0.29
400,acajutla,SV,0.0,1580000298,79,13.59,-89.83,75.00,75.00,6.13
420,robertsport,LR,0.0,1580000308,83,6.75,-11.37,81.18,81.18,8.48


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = ideal_cities
hotel_df["Hotel"] = ""
hotel_df["Hotel Lat"] = ""
hotel_df["Hotel Lng"] = ""


for index, row in hotel_df.iterrows():

    # geocoordinates
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "Hotels"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    try: 
        firstResult = response.json()["results"][0]

        hotel_df.loc[index, "Hotel"]    = firstResult["name"]
        hotel_df.loc[index, "Hotel Lat"] = firstResult["geometry"]["location"]['lat']
        hotel_df.loc[index, "Hotel Lng"] = firstResult["geometry"]["location"]['lng']
        
    except: 
        hotel_df.loc[index,"Hotel"] = "No Hotels Found"
        hotel_df.loc[index,"Hotel Lat"] = "N/A"
        hotel_df.loc[index,"Hotel Lng"] = "N/A"
hotel_df



,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Wind Speed,Hotel,Hotel Lat,Hotel Lng
31,arraial do cabo,BR,0.0,1579999938,82,-22.97,-42.03,75.78,75.78,8.32,Orlanova Hotel,-22.9614,-42.0215
120,takapau,NZ,0.0,1580000163,68,-40.03,176.35,84.00,75.99,4.00,No Hotels Found,N/A,N/A
147,port macquarie,AU,0.0,1580000177,55,-31.43,152.92,97.00,86.00,5.82,Sails Port Macquarie by Rydges,-31.4271,152.899
193,karratha,AU,0.0,1580000199,74,-20.74,116.85,85.17,85.17,4.34,Karratha International Hotel,-20.7375,116.842
295,bella vista,PY,0.0,1580000248,53,-22.13,-56.52,74.61,74.61,0.29,Pousada Bem Estar,-22.1029,-56.5255
400,acajutla,SV,0.0,1580000298,79,13.59,-89.83,75.00,75.00,6.13,Villavela Hotel Boutique,13.5887,-89.8342
420,robertsport,LR,0.0,1580000308,83,6.75,-11.37,81.18,81.18,8.48,Nana's Lodge,6.75036,-11.3805


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))